**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from class_utils import error_histogram
import matplotlib.pyplot as plt

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("sigmoid_regression_data.csv"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Regresia a predspracovanie dát pre ňu

Ako ďalší príklad ukážeme, ako sa dá realizovať jednoduchá regresná úloha. Ako regresnú metódu použijeme (rovnako ako v prípade klasifikácie) metódu $k$ najbližších susedov (angl. $k$ nearest neighbours; KNN). V prípade klasifikácie sa predikcia určila hlasovaním medzi $k$ najbližšími susedmi. V prípade regresie sa zase predikcia vyráta jednoducho ako priemer z hodnôt $k$ najbližších susedov.

### Načítanie dátovej množiny

Príklad začneme, ako obvykle, načítaním dátovej množiny, ktorá má znovu podobu CSV súboru: v tomto prípade len s dvoma stĺpcami. Prvý z nich, $x$, predstavuje vstup nášho modelu a druhý $y$, reprezentuje požadovaný výstup.



In [ ]:
df = pd.read_csv("data/sigmoid_regression_data.csv")
df.head()

### Stratifikácia pri delení dátovej množiny

Ďalším krokom je samozrejme rozdelenie dátovej množiny na tréningovú a testovaciu časť. Pripomeňme, že pri klasifikácii sme v rámci delenia dátovej množiny používali stratifikáciu podľa triedy – aby sme zabezpečili, že pomer tried v tréningovej aj v testovacej časti množiny zostane podobný ako bol v pôvodnej dátovej množine.

V prípade regresie sa samozrejme nedá spraviť to isté – alebo aspoň nie tak priamočiaro ako pri klasifikácii – keďže výstupné hodnoty sú spojité. V niektorých prípadoch však môže byť stále žiaduce nejaký druh stratifikácie vykonať – najmä ak je dátová množina malá, ako je to aj v našom prípade. Inak sa môže stať, že tréningová a testovacia množina nebudú reprezentatívne.

Ako by taký problém vyzeral sa dá ukázať ľahko: stačí našu dátovú množinu rozdeliť štandardne, bez použitia stratifikácie, a vizualizovať v tom istom grafe tréningové aj testovacie dáta:



In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=4)

In [ ]:
plt.scatter(df_train['x'], df_train['y'], marker='x', label="tréningové dáta")
plt.scatter(df_test['x'], df_test['y'], c='r', label="testovacie dáta")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()
plt.savefig("output/regression_split_plain.pdf", bbox_inches='tight', pad_inches=0)

Z grafu by malo byť vidno, že testovacie dáta ani zďaleka nepokrývajú celý priestor. Ako teda stratifikáciu aplikovať, aby sme takému problému zamedzili? Keďže stĺpec podľa ktorého stratifikujeme by mal obsahovať diskrétne hodnoty, najjednoduchšie je samozrejme náš spojitý stĺpec diskretizovať – napr. pomocou transformátora `KBinsDiscretizer` z balíčka `scikit-learn`. Samotná diskretizácia môže vyzerať napríklad nasledovne:



In [ ]:
kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

Podľa diskretizovaného stĺpca `y_stratify` už môžeme stratifikovať:



In [ ]:
df_train, df_test = train_test_split(df, stratify=y_stratify,
                                 test_size=0.3, random_state=4)

Keď vizualizujeme výsledky takéhoto delenia, mali by sme vidieť, že body sú rozdelené medzi tréningovú a testovaciu množinu výrazne rovnomernejšie:



In [ ]:
plt.scatter(df_train['x'], df_train['y'], marker='x', label="training data")
plt.scatter(df_test['x'], df_test['y'], c='r', label="testing data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()
plt.savefig("output/regression_split_stratif.pdf", bbox_inches='tight', pad_inches=0)

---
### Úloha 1: Zostavenie pipeline

**Dalším krokom je, ako zvyčajne, zostavenie pipeline na predspracovanie dát. Najprv určte, ktoré vstupné stĺpce sú kategorické a ktoré numerické a následne vytvorte rovnaký pipeline objekt, aký sme používali v predchádzajúcich príkladoch.** 

POZN.: Výstupný stĺpec nebude potrebné predspracovať.

---


In [ ]:
categorical_inputs = [           ]  # ----

numeric_inputs = [               ]  # ----

output = 'y'

In [ ]:
input_preproc = make_column_transformer(
    
    
    
    # ----


    

### Predspracovanie dát

Dáta predspracujeme tradičným spôsobom:



In [ ]:
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values

### Tréning modelu

Model sa trénuje pomocou unifikovaného rozhrania – zmení sa len jeho typ. Namiesto `KNeighborsClassifier` používame `KNeighborsRegressor`.



In [ ]:
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train, Y_train)

### Testovanie modelu

Nakoniec správanie sa modelu overíme na testovacích dátach. V prípade regresie sa samozrejme ako metrika úspešnosti nepoužíva správnosť – regresia má svoje vlastné ukazovatele. Medzi tie najčastejšie používané patria **stredná kvadratická chyba**  (angl. mean squared error; MSE):
\begin{equation}
MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2.
\end{equation}
kde $y_i$ je požadovaný výstup a $\hat{y}_i$ skutočný výstup modelu pre vzorku $i$ z dátovej množiny a $N$ je celkový počet vzoriek v dátovej množine.

a **stredná absolútna chyba**  (angl. mean absolute error; MAE):
\begin{equation}
MAE = \frac{1}{N} \sum_{i=1}^{N} |y_i - \hat{y}_i|.
\end{equation}

Stredná kvadratická chyba je citlivejšia na veľké chyby (kvôli druhej mocnine). Výhodou strednej absolútnej chyby je, že má rovnakú škálu ako dáta, takže je ju ľahšie interpretovať (ale aj stredná kvadratická chyba sa dá odmocniť). Existujú samozrejme aj ďalšie ukazovatele (napr. percentuálne chyby, koeficient determinácie a pod.), ale tým sa v tomto notebook-u venovať nebudeme.

Použime teda vytvorený regresný model a vypočítajme predikcie na testovacích dátach.



In [ ]:
y_test = model.predict(X_test)

Vypočítame a zobrazíme strednú kvadratickú a strednú absolútnu chybu. Pripomeňme, že výstupy sú približne z rozsahu od 0 po 1.



In [ ]:
mse = mean_squared_error(Y_test, y_test)
mae = mean_absolute_error(Y_test, y_test)

print("MSE: {}".format(mse))
print("MAE: {}".format(mae))

Pre plnšiu predstavu o tom, ako sú chyby rozdelené môžeme vizualizovať aj celé ich rozdelenie pomocou histogramu výstupov a chýb. Výstupy pre tento graf štandardizujeme, aby boli histogram výstupov a histogram chýb zarovnané. Rovnako aj strednú absolútnu chybu počítame znovu zo škálovaných dát, aby bola v tej istej mierke.



In [ ]:
plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)
plt.savefig("output/error_output_histogram.pdf", bbox_inches='tight', pad_inches=0)

### Vizualizácia regresnej závislosti

Keďže naša regresná závislosť je len 2-rozmerná, môžeme ju ľahko vizualizovať v grafe a zhodnotiť kvalitu výsledkov aj vizuálne.



In [ ]:
x_min = min(np.min(X_train), np.min(X_test))
x_max = max(np.max(X_train), np.max(X_test))

xx = np.linspace(x_min, x_max, 250).reshape((-1, 1))
yy = model.predict(xx)

plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")

plt.plot(xx, yy, label="regression curve")

plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

plt.savefig("output/knn_regression.pdf", bbox_inches="tight", pad_inches=0)

Jedna vlastnosť, ktorú by z grafu malo byť vidno, je, že regresná závislosť nie je hladká. Je to samozrejme preto, že existujú oblasti, kde všetky body majú tých istých najbližších susedov a výstup je preto v rámci nich necitlivý na zmenu vstupu. V neskorších notebook-och sa budeme venovať aj iným typom modelov, pomocou ktorých bude možné dosiahnuť lepšie výsledky.

